In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice

In [2]:
plate = 13
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:25]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

In [3]:
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
class Experiment():
    def __init__(self,plate):
        self.plate = plate
        self.path_param = '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat'
    def load(self,dates,local=False,pickle=False,raw=False):
        self.dates=dates
        self.raw=raw
        if local:
            paths=[f'Data/graph_{date}_{self.plate}_full_labeled.csv' for date in dates]
            paths_connection = [f'Data/connection_{date}_{self.plate}.csv' for date in dates[:-1]]
        else:
            paths = [get_path(date,self.plate,True,extension='_full_labeled.csv') for date in dates]
            paths_connection = [get_path(date,self.plate,True,extension='_connection.csv') for date in dates[:-1]]
        if raw:
            nx_graph_poss=[]
            for date in dates:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,self.plate,True,extension='_raw_aligned_skeleton.csv'),
                                                converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))
        else:
            nx_graph_poss=[]
            for path in paths:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(path,
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,
                                                'pixel_list' : ast.literal_eval}),labeled=True))
        from_tip_growth_pattern=[]
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean=[]
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected=[len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        skeletons=[]
        for nx_graph in nx_graph_clean:
            skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions=poss
        self.nx_graph=nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
        self.skeletons=skeletons
        self.hyphaes=None
        labels = {node for g in self.nx_graph for node in g}
        self.nodes=[]
        for label in labels:
            self.nodes.append(Node(label,self))
        
    def copy(self,experiment):
        self.positions=experiment.positions
        self.nx_graph=experiment.nx_graph
        self.connections = experiment.connections
        self.growth_patterns = experiment.growth_patterns
        self.skeletons=experiment.skeletons
        self.dates=experiment.dates
        self.plate=experiment.plate
        self.nodes = None
        self.hyphaes=None
    def save(self,path=f'Data/'):
        from_tip_growth_pattern_tab=[]
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(pd.read_csv(get_path(date,self.plate,True,extension='_connection.csv')))
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(path+f'connection_{date}_{self.plate}.csv')
        tabs_labeled=[]
        for date in self.dates:
            tabs_labeled.append(pd.read_csv(get_path(date,self.plate,True,extension='_full_labeled.csv'),
                                        converters={'origin_pos' : transform_list,
                                                    'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path+f'graph_{date}_{self.plate}_full_labeled.csv')
            sio.savemat(path+f'graph_{date}_{self.plate}_full_labeled.mat', {name: col.values for name, col in tabs_labeled[i].items()})
    def pickle_save(self):
        pickle.dump(self,open( f'Data/'+f"experiment_{self.plate}.pick", "wb" ))
    def pickle_load(self):
        self = pickle.load( open( f'Data/'+f"experiment_{self.plate}.pick", "rb" ) )
    def get_node(self,label):
        return(Node(label,self))
    def get_edge(self,begin,end):
        return(Edge(begin,end,self))
    def compress_skeleton(self,t,factor):
        shape=self.skeletons[t].shape
        final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
        for pixel in self.skeletons[t].keys():
            x=min(round(pixel[0]/factor),shape[0]//factor-1)
            y=min(round(pixel[1]/factor),shape[1]//factor-1)
            final_picture[x,y]+=1
        return(final_picture>=1)
    def load_images(self,t):
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        for name in tileconfig[0]:
            im=imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name)
            np.save('Temp/'+name,im)
    def get_growing_tips(self,t,threshold=80):
        growths = {tip : sum([len(branch) for branch in self.growth_patterns[t][tip]]) for tip in self.growth_patterns[t].keys()}
        growing_tips = [node for node in growths.keys() if growths[node]>=threshold]
        return(growing_tips)
    def pinpoint_anastomosis(self,t):
        nx_graph_tm1 = self.nx_graph[t]
        nx_grapht = self.nx_graph[t+1]
        from_tip = self.connections[t]
        pos_tm1 = self.positions[t]
        anastomosis=[]
        origins=[]
        tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
        number_anastomosis = 0
        def dist_branch(node,nx_graph,pos):
            mini=np.inf
            for edge in nx_graph.edges:
                pixel_list=nx_graph.get_edge_data(*edge)['pixel_list']
                if np.linalg.norm(np.array(pixel_list[0])-np.array(pos[node]))<=5000:
                    distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos[node]),axis=1))
                    if distance<mini:
                        mini=distance
            return(mini)
        def count_neighbors_is_from_root(equ_list,nx_graph,root):
            count=0
            for neighbor in nx_graph.neighbors(root):
                if neighbor in equ_list:
                    count+=1
            return(count)
        for tip in tips:
    #         print(tip)
            consequence = from_tip[tip]
            for node in consequence:
                if node in nx_grapht.nodes and nx_grapht.degree(node)>=3 and count_neighbors_is_from_root(consequence,nx_grapht,node)<2:
    #                 if node==2753:
    #                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
    #                     print(list(nx_grapht.neighbors(node)))
                    anastomosis.append(node)
                    origins.append(tip)
                    number_anastomosis+=1
            if tip not in nx_grapht.nodes and dist_branch(tip,nx_grapht,pos_tm1)<=30 and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['weight']>=20:
                origins.append(tip)
                number_anastomosis+=1/2
        return(anastomosis,origins,number_anastomosis)
    def find_image_pos(self,ys,xs,t,local=False):
        params=read_mat(self.path_param)
        x1=params['x1']
        x2=params['x2']
        y0=params['y0']
        Sf=params['Sf']
        L0center=params['L0center']
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        findingbaits = read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/FindingBaits.mat')
        Rcenter=findingbaits['Rcenter']
        Rradii = findingbaits['Rradii']
        Lcenter = findingbaits['Lcenter']
        t = findingbaits['t']
        Lradii = findingbaits['Lradii']
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        xs_yss=list(tileconfig[2])
        xes= [xs_ys[0] for xs_ys in xs_yss]
        yes= [xs_ys[1] for xs_ys in xs_yss]
        cmin=np.min(xes)
        cmax=np.max(xes)
        rmin = np.min(yes)
        rmax = np.max(yes)
        S0     = [rmax+3000+abs(rmin),cmax+4096+abs(cmin)]
        theta  = np.arctan((Rcenter[1]-Lcenter[1])/(Rcenter[0]-Lcenter[0]))
        Sry    =  S0[0]*np.cos(abs(theta))+S0[1]*np.sin(abs(theta))
        C = np.tan(-np.arctan((ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/
                              (xs + (x2+x1)/2 - (Sf[0]-y0) - 1 - L0center[0] + np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.cos(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))) - theta)
        ximg = - (ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                  *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(np.sqrt(1+C**2)*np.sin(np.arctan(C)+theta)) 
        yimg = S0[0] - C*ximg
        def find(xsub,ysub,x,y):
            indexes=[]
            for i in range(len(xsub)):
                if x>=xsub[i]-cmin and x<xsub[i]-cmin+4096 and y>=ysub[i]-rmin and y < ysub[i]-rmin+3000:
                    indexes.append(i)
            return(indexes)
        indsImg = find(xes,yes,ximg,yimg)
        possImg  = [ximg-np.array(xes)[indsImg]+cmin+1, yimg-np.array(yes)[indsImg]+rmin+1]
        paths=[]
        for index in indsImg:
            if local:
                paths.append('Temp/'+tileconfig[0][index]+'.npy')
            else:
                paths.append('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+tileconfig[0][index])
        if local:
             ims = [np.load(path) for path in paths]
        else:
            ims = [imageio.imread(path) for path in paths]
        return(ims,possImg)
    def plot(self,ts,node_lists=[], shift=(0,0),compress=5,save='',time=None):
        global check
        right = 0.90
        top = 0.90
        if len(save)>=1:
            fig=plt.figure(figsize=(14,12))
            size = 10
        else:
            fig = plt.figure()
            size = 5
        ax = fig.add_subplot(111)
        scale=1/len(ts)
        power=len(ts)
        compressed_images=[]
        for i,t in enumerate(ts):
            compressed_images.append(self.compress_skeleton(t,compress))
        visibility = [True for t in ts]
        final_image=scale*compressed_images[0]*visibility[0]
        for i,compressed_image in enumerate(compressed_images[1:]):
            final_image=np.power(final_image**power+((i+2)*scale*compressed_image*visibility[i+1])**power,1/power)
        l1=ax.imshow(final_image, cmap='plasma',interpolation='none',vmin=0,vmax=1)
        rax = plt.axes([0.05, 0.4, 0.1, 0.15])
        labels=[f'{4*t}h' for t in ts]
        check = CheckButtons(rax, labels, visibility)
        def func(label):
            index = labels.index(label)
            visibility[index]= not visibility[index]
            final_image=visibility[0]*scale*compressed_images[0]
            for i,compressed_image in enumerate(compressed_images[1:]):
                final_image=np.power(final_image**power+visibility[i+1]*((i+2)*scale*compressed_image)**power,1/power)
            l1.set_data(final_image)
            plt.draw()

        check.on_clicked(func)
        if len(node_lists)>0:
            for i,node_list in enumerate(node_lists):
                grey=(i+1)/len(labels)
                bbox = dict(boxstyle="circle", fc=colors.rgb2hex((grey,grey,grey)))
#             ax.text(right, top, time,
#                 horizontalalignment='right',
#                 verticalalignment='bottom',
#                 transform=ax.transAxes,color='white')
                for node in node_list:
#                     print(self.positions[ts[i]])
                    if node in self.positions[ts[i]].keys():
                        t = ax.text((self.positions[ts[i]][node][1]-shift[1])//compress, (self.positions[ts[i]][node][0]-shift[0])//compress, str(node), ha="center", va="center",
                                    size=size,
                                    bbox=bbox)
        if len(save)>=1:
            plt.savefig(save)
            plt.close(fig)
        else:
            plt.show()
        

class Node():
    def __init__(self,label,experiment):
        self.experiment = experiment
        self.label = label
    def __eq__(self, other): 
        return(self.label==other.label)
    def __repr__(self):
        return (f'Node({self.label})')
    def __str__(self):
        return str(self.label)
    def __hash__(self):
        return self.label
    def neighbours(self,t):
        return([self.experiment.get_node(node) for node in self.experiment.nx_graph[t].neighbors(self.label)])
    def is_in(self,t):
        return(self.label in self.experiment.nx_graph[t].nodes)
    def degree(self,t):
        return(self.experiment.nx_graph[t].degree(self.label))
    def edges(self,t):
        return([self.experiment.get_edge(self,neighbour) for neighbour in self.neighbours(t)])
    def pos(self,t):
        return(self.experiment.positions[t][self.label])
    def ts(self):
        return([t for t in range(len(self.experiment.nx_graph)) if self.is_in(t)])

class Edge():
    def __init__(self,begin,end,experiment):
        self.begin=begin
        self.end = end
        self.experiment = experiment
    def __repr__(self):
        return (f'Edge({self.begin},{self.end})')
    def __str__(self):
        return str((self.begin,self.end))
    def is_in(self,t):
        return((self.begin.label,self.end.label) in self.experiment.nx_graph[t].edges)
    def pixel_list(self,t):
        return(orient(self.experiment.nx_graph[t].get_edge_data(self.begin.label,self.end.label)['pixel_list'],self.begin.pos(t)))
    def orientation_whole(self,t):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
    def orientation_begin(self,t,length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[min(length,len(pixel_list)-1)]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
    def orientation_end(self,t,length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[max(0,len(pixel_list)-1-length)]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
        

class Hyphae():
    def __init__(self,tip):
        self.experiment = tip.experiment
        self.ts = tip.ts()
        self.end = tip
        self.root = None
        self.mother = None
    def __repr__(self):
        return (f'Hyphae({self.end},{self.root})')
    def __str__(self):
        return str((self.end,self.root))
    def get_edges(self,t,length=20):
        first_neighbour = self.end.neighbours(t)[0]
        llast_node = self.end
        ccurrent_node = first_neighbour
        def move_hyphae(last_node,current_node):
            current_edge = Edge(last_node,current_node,current_node.experiment)
            moving_on_hyphae = True
            edges = [current_edge]
            nodes = [last_node,current_node]
            while moving_on_hyphae:
#                 print ('moving',current_node)
                if current_node.degree(t)<2:
#                     print(current_node.degree(t),current_node)
                    return(edges,nodes,current_node)
                else:
                    maxi=-np.inf
                    orientation = current_edge.orientation_end(t,length)
                    for neighbours_t in current_node.neighbours(t):
    #                     print (neighbours_t)
                        candidate_edge = Edge(current_node,neighbours_t,current_node.experiment)
                        orientation_candidate = candidate_edge.orientation_begin(t,length)
                        angle = np.cos((orientation-orientation_candidate)/360*2*np.pi)
                        if angle>maxi:
                            maxi=angle
                            next_node_candidate=neighbours_t
#                     print(maxi,next_node_candidate)
                    candidate_edge = Edge(current_node,next_node_candidate,current_node.experiment)
                    orientation_candidate = candidate_edge.orientation_begin(t,length)
                    maxi_compet=-np.inf
#                     print('compet')
                    for neighbours_t in current_node.neighbours(t):
                        if neighbours_t!=last_node:
                            competitor_edge = Edge(neighbours_t,current_node,current_node.experiment)
                            orientation_competitor = competitor_edge.orientation_end(t,length)
                            angle = np.cos((orientation_competitor-orientation_candidate)/360*2*np.pi)
                            if angle>maxi_compet:
                                maxi_compet=angle
                                competitor = neighbours_t
#                             print(neighbours_t,angle)
#                     print(maxi_compet,competitor)
                    if maxi_compet>maxi:
                        moving_on_hyphae =False
                    else:
                        last_node,current_node = current_node,next_node_candidate
                        current_edge = Edge(last_node,current_node,current_node.experiment)
                        edges.append(current_edge)
                        nodes.append(current_node)
            return(edges,nodes,competitor)
        moving = True
        edges = []
        nodes = []
        c= move_hyphae(llast_node,ccurrent_node)
        edges += c[0]
        nodes += c[1]
        competitor = c[2]
#         while moving:
#             c= move_hyphae(llast_node,ccurrent_node)
#             edges += c[0]
#             nodes += c[1]
#             competitor = c[2]
# #             print('moving back', nodes[-1],competitor)
#             move_backward = move_hyphae(nodes[-1],competitor)
#             end_node_move_backward = move_backward[1][-1]
#             if end_node_move_backward in nodes:
# #                 print('restarting',competitor,nodes[-1])
#                 llast_node,ccurrent_node = c[2],nodes[-1]
#             else:
#                 moving=False
        root = nodes[-1]
        edges = edges
        nodes = nodes
        return(root,edges,nodes)
    def get_nodes_within(self,t):
        nodes = nx.shortest_path(self.experiment.nx_graph[t],source=self.root.label,target=self.end.label,weight="weight")
        edges = [Edge(self.experiment.get_node(nodes[i]),self.experiment.get_node(nodes[i+1]),self.experiment) for i in range(len(nodes)-1)]
        return(nodes,edges)
    def get_mother(self):
        candidate_mother=[]
        for hyphae in self.experiment.hyphaes:
            for t in hyphae.ts:
                if self.root in hyphae.get_nodes_within(t):
                    candidate_mother.append(hyphae)
        self.mother = candidate_mother

In [4]:
exp= Experiment(13)
exp.load(dates,local=True,raw=False)
# exp.pickle_save()

In [5]:
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

In [236]:
len(problems)

78

In [237]:
len(hyphaes)

3374

In [156]:
problem_hyphae = []
distances = []
for hyphae in hyphaes:
    root = hyphae.root
    end = hyphae.end
    for hyphae2 in hyphaes:
        end2 = hyphae2.end
        if hyphae2.end != hyphae.end and hyphae2.root==root and end.ts()[0]!=end2.ts()[-1]+1 and end.ts()[-1]+1!=end2.ts()[0] and len(end.ts())>=2 and len(end2.ts())>=2:
            distance
            distances.append(np.linalg.norm(hyphae.end.pos(hyphae.end.ts()[-1])-hyphae2.end.pos(hyphae2.end.ts()[-1])))
            problem_hyphae.append((hyphae,hyphae2))

In [157]:
problem_roots = {hyphaec[0].root for hyphaec in problem_hyphae}

In [251]:
problem=choice(hyphaes)

In [252]:
problem.get_nodes_within(problem.end.ts()[4])

[Edge(9789,6906),
 Edge(6906,7105),
 Edge(7105,11753),
 Edge(11753,8671),
 Edge(8671,9287)]

In [201]:
labels=[problem[0].root.label,problem[0].end.label,problem[1].end.label]

In [275]:
exp.plot([problem.end.ts()[4]],[[9789,6906,7105,11753,8671]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [204]:
plt.close('all')

In [128]:
node=choice(to_look)
print(exp.get_node(node).degree(tis[node][0]))
plt.close('all')

3


In [114]:
to_look_degree1 = [node for node in to_look if exp.get_node(node).degree(tis[node][0])==1]
node=choice(to_look_degree1)
plt.close('all')

In [129]:
exp.plot([tis[node][0],tis[node][0]+1],node_lists=[[node]],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
exp_clean.plot([tis[node][0],tis[node][0]+1],node_lists=[[node]],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [144]:
exp.plot([9,10],[[6924,6922,2758]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
exp_clean.plot([11],[[7482,7639,7146,6924]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
plt.close('all')

In [48]:
exp.plot([0],[[859]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
to_look,tis = clean_exp(exp)

In [48]:
9726 in to_look

False

In [51]:
exp.get_node(9726).pos(15)

array([15239, 34643])

In [56]:
ims,poss=exp.find_image_pos(15239,34643,15,local=False)
i=1
plot_t_tp1([0],[],{0:(poss[1][i],poss[0][i])},None,ims[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
hyphae.edges

[Edge(7482,8876),
 Edge(8876,6882),
 Edge(6882,10102),
 Edge(10102,9726),
 Edge(9726,9879),
 Edge(9879,7543),
 Edge(7543,9937),
 Edge(9937,9937)]

In [43]:
exp.get_node(2758).neighbours(15)

[Node(3225), Node(6935), Node(1042)]

In [59]:
Edge(exp.get_node(8876),exp.get_node(6882),exp).orientation_begin(15)

168.69006752597983

In [63]:
Edge(exp.get_node(8853),exp.get_node(8876),exp).orientation_begin(15)

-145.00797980144137

In [11]:
from collections import Counter
def get_hyphae(experiment):
    tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1]
    problems=[]
    hyphaes=[]
#     for i in range(20):
    for i,tip in enumerate(tips):
        if i%200==0:
            print(i/len(tips))
#         tip = choice(tips)
        hyphae = Hyphae(tip)
        roots = []
        for t in tip.ts():
#             print(t,tip)
            if tip.degree(t)==1:
                root,edges,nodes = hyphae.get_edges(t,50)
                roots.append(root)
        occurence_count = Counter(roots)
        if len(occurence_count.values())>=2 and occurence_count.most_common(2)[0][0]!=roots[0] and occurence_count.most_common(2)[1][1]/occurence_count.most_common(2)[0][1]>=0.75:
#             print(roots[0])
#             print(tip,occurence_count)
            problems.append(tip)
        else:
            hyphae.root = occurence_count.most_common(2)[0][0]
            hyphaes.append(hyphae)
    return(hyphaes,problems)
def reconnect_degree_2(nx_graph,pos):
    degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]
    while len(degree_2_nodes)>0:
        node = degree_2_nodes.pop()
        neighbours = list(nx_graph.neighbors(node))
        right_n = neighbours[0]
        left_n = neighbours[1]
        right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
        left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
        if np.any(right_edge[0]!=pos[node]):
            right_edge = list(reversed(right_edge))
        if np.any(left_edge[-1]!=pos[node]):
            left_edge = list(reversed(left_edge))
        pixel_list = left_edge+right_edge[1:]
        info={'weight':len(pixel_list),'pixel_list':pixel_list}
        if right_n!=left_n:
            connection_data=nx_graph.get_edge_data(right_n,left_n)
            if connection_data is None or connection_data['weight']>=info['weight']:
                if not connection_data is None:
                    nx_graph.remove_edge(right_n,left_n)
                nx_graph.add_edges_from([(right_n,left_n,info)])
        nx_graph.remove_node(node)
        degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]

def clean_exp(experiment):
    ts={}
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
    exp_clean = Experiment(experiment.plate)
    exp_clean.copy(experiment)
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes=[]
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    for node in experiment.nodes:
        ts[node.label]=[]
        for t in range(len(nx_graph_cleans)):
            if node.is_in(t):
                ts[node.label].append(t)
    hyphaes,problems = get_hyphae(exp_clean)
    to_remove=[]
    safe_nodes = set()
    for hyphae in hyphaes:
        for t in hyphae.end.ts():
            if t in hyphae.root.ts():
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if len(ts[node.label])==1 and ts[node.label][0]!=len(experiment.nx_graph)-1 and node.label not in safe_nodes:
            to_remove.append(node.label)
#     return(to_remove)
    print('removing ',len(to_remove),' nodes')
    for node in to_remove:
        t= ts[node][0]
        pos = experiment.positions[t]
        nx_graph_clean = nx_graph_cleans[t]
        if nx_graph_clean.degree(node)<=2:
            continue
#         print('cleaning',node)
        neighbours = list(nx_graph_clean.neighbors(node))
        candidate_to_fuse=[]
        weight_candidate=[]
        for neighbour in neighbours:
            if nx_graph_clean.degree(neighbour)>=3:
                candidate_to_fuse.append(neighbour)
                weight_candidate.append(len(nx_graph_clean.get_edge_data(node,neighbour)['pixel_list']))
        if len(weight_candidate)==0:
            continue
        node_to_fuse=candidate_to_fuse[np.argmin(weight_candidate)]
        if nx_graph_clean.degree(node)==1 and node_to_fuse not in to_remove:
            print(pos[node])
            continue
        for neighbour in neighbours:
            right_n = node_to_fuse
            left_n = neighbour
            right_edge = nx_graph_clean.get_edge_data(node,right_n)['pixel_list']
            left_edge = nx_graph_clean.get_edge_data(node,left_n)['pixel_list']
            if np.any(right_edge[0]!=pos[node]):
                right_edge = list(reversed(right_edge))
            if np.any(left_edge[-1]!=pos[node]):
                left_edge = list(reversed(left_edge))
            pixel_list = left_edge+right_edge[1:]
            info={'weight':len(pixel_list),'pixel_list':pixel_list}
            if right_n!=left_n:
                connection_data=nx_graph_clean.get_edge_data(right_n,left_n)
                if connection_data is None or connection_data['weight']>=info['weight']:
                    if not connection_data is None:
                        nx_graph_clean.remove_edge(right_n,left_n)
                    nx_graph_clean.add_edges_from([(right_n,left_n,info)])
        nx_graph_clean.remove_node(node)
        nx_graph_cleans[t]=nx_graph_clean
    for t,nx_graph in enumerate(nx_graph_cleans):
        pos = experiment.positions[t]
        reconnect_degree_2(nx_graph,pos)
    nx_graph_clean=[]
    for graph in nx_graph_cleans:
        S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
        len_connected=[len(nx_graph.nodes) for nx_graph in S]
        nx_graph_clean.append(S[np.argmax(len_connected)])
    skeletons=[]
    for nx_graph in nx_graph_clean:
        skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
    exp_clean.nx_graph=nx_graph_clean
    exp_clean.skeletons=skeletons
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes=[]
    exp_clean.hyphaes = hyphaes
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    return(exp_clean)

In [12]:
exp_clean= clean_exp(exp)

0.0
0.05793742757821553
0.11587485515643106
0.17381228273464658
0.23174971031286212
0.28968713789107764
0.34762456546929316
0.4055619930475087
0.46349942062572425
0.5214368482039398
0.5793742757821553
0.6373117033603708
0.6952491309385863
0.7531865585168018
0.8111239860950173
0.8690614136732329
0.9269988412514485
0.984936268829664
removing  2863  nodes


In [414]:
hyphae = choice(exp_clean.hyphaes)
candidate,problems = get_mother(hyphae)

In [460]:
for i in range(10):
    hyphae = choice(exp_clean.hyphaes)
    candidate,problems = get_mother(hyphae)
    print(candidate)
    if len(candidate)>=2:
        break


[]
[Hyphae(1874,3529)]
[Hyphae(7448,7464)]
[Hyphae(79,827), Hyphae(12478,827)]


In [451]:
def get_mother(hyph):
    candidate_mother=[]
    problem=[]
    for hyphae in hyph.experiment.hyphaes:
        if hyphae.end != hyph.end:
            found = False
            for t in hyphae.ts():
                if hyphae.root.is_in(t):
                    if hyph.root.label in hyphae.get_nodes_within(t)[0] and hyph.root.label != hyphae.root.label and found == False and hyphae.end.pos(t)[0]<19500:
                        candidate_mother.append(hyphae)
                        found = True
                else:
                    problem.append((hyphae,t))
    return(candidate_mother,problem)

In [485]:
def get_disapearing_roots(hyphaes):
    problems=[]
    for hyph in hyphaes:
        if len(hyph.root.ts())<len(hyph.ts()):
            problems.append(hyph)
    return(problems)

In [486]:
prob = get_disapearing_roots(exp_clean.hyphaes)

In [487]:
len(prob)

378

In [505]:
hyphae=choice(prob)

In [ ]:
hyphae.root.ts(),hyphae.ts()

In [470]:
def resolve_ambiguity(hyphaes):
    ambiguities=[]
    for hyph in hyphaes:
        t0=hyph.ts()[0]
        nodes = hyph.get_nodes_within(t0)
        root,first = nodes[0],nodes[1]
        for hyph2 in hyphaes:
            if hyph2.end != hyph.end:
                t1=hyph2.ts()[0]
                if t1
                nodes2 = hyph2.get_nodes_within(t0)
                if root in nodes2 and first in nodes2:
                    ambiguities.append(hyph,hyph2)
    return(ambiguities)

In [ ]:
resolve_ambiguity(exp_clean.hyphaes)

In [ ]:
candidate[1].ts(),candidate[0].ts()

In [462]:
tis = [20,21]

In [447]:
candidate[0].end.pos(2)

array([19648, 10906])

In [499]:
hyphae.root

Node(3228)

In [506]:
hyphae.root.ts(),hyphae.ts()

([22, 23], [21, 22, 23])

In [507]:
exp_clean.plot([21,22],[[hyphae.root.label,hyphae.end.label]*2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [465]:
exp_clean.plot([tis],[candidate[0].get_nodes_within(tis[0])[0]+candidate[1].get_nodes_within(tis[0])[0]+[hyphae.end.label,hyphae.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [494]:
plt.close('all')

In [ ]:
exp_clean.hyphaes[0].experiment.hyphaes

In [274]:
problems = clean_exp2(exp)[0]

0.0
0.05793742757821553
0.11587485515643106
0.17381228273464658
0.23174971031286212
0.28968713789107764
0.34762456546929316
0.4055619930475087
0.46349942062572425
0.5214368482039398
0.5793742757821553
0.6373117033603708
0.6952491309385863
0.7531865585168018
0.8111239860950173
0.8690614136732329
0.9269988412514485
0.984936268829664


In [273]:
def clean_exp2(experiment):
    ts={}
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
    for node in experiment.nodes:
        ts[node.label]=[]
        for t in range(len(nx_graph_cleans)):
            if node.is_in(t):
                ts[node.label].append(t)
    hyphaes,problems = get_hyphae(experiment)
    to_remove=[]
    safe_nodes = set()
    for hyphae in hyphaes:
        for t in hyphae.end.ts():
            if t in hyphae.root.ts():
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if len(ts[node.label])==1 and ts[node.label][0]!=len(experiment.nx_graph)-1 and node.label not in safe_nodes:
            to_remove.append(node.label)
    return(to_remove,ts)

In [321]:
node = choice(problems)
plt.close('all')

In [322]:
exp.plot([exp.get_node(node).ts()[0],exp.get_node(node).ts()[0]+1],[[node]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [323]:
exp_clean.plot([exp.get_node(node).ts()[0],exp.get_node(node).ts()[0]+1],[[node]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
occurence_count = Counter([1,1,1,1,2,2,2,3,3,3]) 

In [81]:
occurence_count[1]

4

In [204]:
def clean_degree_4(nx_graph,pos,thresh=20):
    nx_graph_clean = nx.Graph.copy(nx_graph)
    remaining_to_fuse=True
    print(len(nx_graph_clean.nodes))
    while remaining_to_fuse:
        remaining_to_fuse=False
        to_fuse=[]
        for edge in nx_graph_clean.edges:
            if nx_graph_clean.get_edge_data(*edge)['weight']<=thresh and nx_graph_clean.degree(edge[0])==3 and nx_graph_clean.degree(edge[1])==3:
                to_fuse.append(edge)
        nodes_to_fuse = [edge[0] for edge in to_fuse]+[edge[1] for edge in to_fuse]
        occurence_count = Counter(nodes_to_fuse) 
        difficult_cases=[]
        for edge in to_fuse:
            node1=edge[0]
            node2=edge[1]
            if occurence_count[node1]!=1 and occurence_count[node2]!=1:
                difficult_cases.append(edge)
            else:
                remaining_to_fuse=True
                if occurence_count[node1]==1:
                    pivot = node2
                    fuser = node1
                else:
                    pivot = node1
                    fuser = node2
                neighbours = list(nx_graph_clean.neighbors(fuser))
                for neighbour in neighbours:
                    right_n = pivot
                    left_n = neighbour
                    right_edge = nx_graph_clean.get_edge_data(fuser,right_n)['pixel_list']
                    left_edge = nx_graph_clean.get_edge_data(fuser,left_n)['pixel_list']
                    if np.any(right_edge[0]!=pos[fuser]):
                        right_edge = list(reversed(right_edge))
                    if np.any(left_edge[-1]!=pos[fuser]):
                        left_edge = list(reversed(left_edge))
                    pixel_list = left_edge+right_edge[1:]
                    info={'weight':len(pixel_list),'pixel_list':pixel_list}
                    if right_n!=left_n:
                        connection_data=nx_graph_clean.get_edge_data(right_n,left_n)
                        if connection_data is None or connection_data['weight']>=info['weight']:
                            if not connection_data is None:
                                nx_graph_clean.remove_edge(right_n,left_n)
                            nx_graph_clean.add_edges_from([(right_n,left_n,info)])
                nx_graph_clean.remove_node(fuser)
        print(len(difficult_cases))
    print(len(nx_graph_clean.nodes))
    return(nx_graph_clean,difficult_cases)

In [205]:
nx_graph_clean,difficult = clean_degree_4(exp_clean.nx_graph[12],exp_clean.positions[12])

4153
37
15
5
0
0
0
0
3814


In [15]:
nodes_to_fuse = [edge[0] for edge in to_fuse]+[edge[1] for edge in to_fuse]
occurence_count = Counter(nodes_to_fuse) 

In [181]:
len(exp.nx_graph[12].edges)

5165

In [185]:
for edge in exp_clean.nx_graph[12].edges:
    if edge[0] not in exp_clean.nx_graph[12].nodes or edge[1] not in exp_clean.nx_graph[12].nodes:
        print(edge)
    

In [186]:
for edge in nx_graph_clean.edges:
    if edge[0] not in nx_graph_clean.nodes or edge[1] not in nx_graph_clean.nodes:
        print(edge)

In [207]:
save_nx = exp_clean.nx_graph[12]
save_skel = exp_clean.skeletons[12]

In [209]:
exp_clean.nx_graph[12] = nx_graph_clean
exp_clean.skeletons[12] = generate_skeleton(nx_graph_clean,dim=(20800, 46000))

In [130]:
len(nx_graph_clean.nodes)

4293

In [92]:
len(difficult)

48

In [64]:
len(exp_clean.nx_graph[15].nodes)

4613

In [65]:
len(exp.nx_graph[15].nodes)

4784

In [46]:
node=choice(occurence_count.most_common(75))[0]
plt.close('all')

In [199]:
edge=choice(difficult)
plt.close('all')

In [200]:
edge[0] in exp_clean.nx_graph[12].nodes, edge[1] in exp_clean.nx_graph[12].nodes

(True, False)

In [201]:
edge[0] in nx_graph_clean.nodes, edge[1] in nx_graph_clean.nodes

(True, True)

In [202]:
exp_clean.get_node(edge[1]).ts(),exp_clean.get_node(edge[0]).ts()

([], [10, 11, 12, 13])

In [210]:
degree_4_nodes = [node for node in nx_graph_clean.nodes if nx_graph_clean.degree(node)>=4]

In [211]:
len(degree_4_nodes)

285

In [221]:
node = choice(degree_4_nodes)
list(nx_graph_clean.neighbors(node))

[3639, 268, 186, 290]

In [222]:
exp_clean.plot([12],[[node]+list(nx_graph_clean.neighbors(node))])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
node=edge[0]

In [140]:
pos = exp_clean.get_node(node).pos(15)
x,y=pos[0],pos[1]
ims,posimg=exp_clean.find_image_pos(x,y,15)

In [1]:
np.mean(ims,axis=0)

NameError: name 'np' is not defined

In [141]:
i=1
plot_t_tp1([0],[],{0 : (posimg[1][i],posimg[0][i])},None,ims[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [513]:
len(exp_clean.nx_graph[15].edges)

5501

In [ ]:
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
class Experiment():
    def __init__(self,plate):
        self.plate = plate
        self.path_param = '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat'
    def load(self,dates,local=False,pickle=False,raw=False):
        self.dates=dates
        self.raw=raw
        if local:
            paths=[f'Data/graph_{date}_{self.plate}_full_labeled.csv' for date in dates]
            paths_connection = [f'Data/connection_{date}_{self.plate}.csv' for date in dates[:-1]]
        else:
            paths = [get_path(date,self.plate,True,extension='_full_labeled.csv') for date in dates]
            paths_connection = [get_path(date,self.plate,True,extension='_connection.csv') for date in dates[:-1]]
        if raw:
            nx_graph_poss=[]
            for date in dates:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,self.plate,True,extension='_raw_aligned_skeleton.csv'),
                                                converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))
        else:
            nx_graph_poss=[]
            for path in paths:
                nx_graph_poss.append(generate_nx_graph(pd.read_csv(path,
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,
                                                'pixel_list' : ast.literal_eval}),labeled=True))
        from_tip_growth_pattern=[]
        for path in paths_connection:
            from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(path)))
        nx_graphs=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
        poss = [nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
        nx_graph_clean=[]
        for graph in nx_graphs:
            S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
            len_connected=[len(nx_graph.nodes) for nx_graph in S]
            nx_graph_clean.append(S[np.argmax(len_connected)])
        skeletons=[]
        for nx_graph in nx_graph_clean:
            skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
        connections = [c[0] for c in from_tip_growth_pattern]
        growth_patterns = [c[1] for c in from_tip_growth_pattern]
        self.positions=poss
        self.nx_graph=nx_graph_clean
        self.connections = connections
        self.growth_patterns = growth_patterns
        self.skeletons=skeletons
        self.hyphaes=None
        labels = {node for g in self.nx_graph for node in g}
        self.nodes=[]
        for label in labels:
            self.nodes.append(Node(label,self))
        
    def copy(self,experiment):
        self.positions=experiment.positions
        self.nx_graph=experiment.nx_graph
        self.connections = experiment.connections
        self.growth_patterns = experiment.growth_patterns
        self.skeletons=experiment.skeletons
        self.dates=experiment.dates
        self.plate=experiment.plate
        self.nodes = None
        self.hyphaes=None
    def save(self,path=f'Data/'):
        from_tip_growth_pattern_tab=[]
        for date in self.dates[:-1]:
            from_tip_growth_pattern_tab.append(pd.read_csv(get_path(date,self.plate,True,extension='_connection.csv')))
        for i, date in enumerate(self.dates[:-1]):
            from_tip_growth_pattern_tab[i].to_csv(path+f'connection_{date}_{self.plate}.csv')
        tabs_labeled=[]
        for date in self.dates:
            tabs_labeled.append(pd.read_csv(get_path(date,self.plate,True,extension='_full_labeled.csv'),
                                        converters={'origin_pos' : transform_list,
                                                    'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))
        for i, date in enumerate(self.dates):
            tabs_labeled[i].to_csv(path+f'graph_{date}_{self.plate}_full_labeled.csv')
            sio.savemat(path+f'graph_{date}_{self.plate}_full_labeled.mat', {name: col.values for name, col in tabs_labeled[i].items()})
    def pickle_save(self):
        pickle.dump(self,open( f'Data/'+f"experiment_{self.plate}.pick", "wb" ))
    def pickle_load(self):
        self = pickle.load( open( f'Data/'+f"experiment_{self.plate}.pick", "rb" ) )
    def get_node(self,label):
        return(Node(label,self))
    def get_edge(self,begin,end):
        return(Edge(begin,end,self))
    def compress_skeleton(self,t,factor):
        shape=self.skeletons[t].shape
        final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
        for pixel in self.skeletons[t].keys():
            x=min(round(pixel[0]/factor),shape[0]//factor-1)
            y=min(round(pixel[1]/factor),shape[1]//factor-1)
            final_picture[x,y]+=1
        return(final_picture>=1)
    def load_images(self,t):
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        for name in tileconfig[0]:
            im=imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name)
            np.save('Temp/'+name,im)
    def get_growing_tips(self,t,threshold=80):
        growths = {tip : sum([len(branch) for branch in self.growth_patterns[t][tip]]) for tip in self.growth_patterns[t].keys()}
        growing_tips = [node for node in growths.keys() if growths[node]>=threshold]
        return(growing_tips)
    def pinpoint_anastomosis(self,t):
        nx_graph_tm1 = self.nx_graph[t]
        nx_grapht = self.nx_graph[t+1]
        from_tip = self.connections[t]
        pos_tm1 = self.positions[t]
        anastomosis=[]
        origins=[]
        tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
        number_anastomosis = 0
        def dist_branch(node,nx_graph,pos):
            mini=np.inf
            for edge in nx_graph.edges:
                pixel_list=nx_graph.get_edge_data(*edge)['pixel_list']
                if np.linalg.norm(np.array(pixel_list[0])-np.array(pos[node]))<=5000:
                    distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos[node]),axis=1))
                    if distance<mini:
                        mini=distance
            return(mini)
        def count_neighbors_is_from_root(equ_list,nx_graph,root):
            count=0
            for neighbor in nx_graph.neighbors(root):
                if neighbor in equ_list:
                    count+=1
            return(count)
        for tip in tips:
    #         print(tip)
            consequence = from_tip[tip]
            for node in consequence:
                if node in nx_grapht.nodes and nx_grapht.degree(node)>=3 and count_neighbors_is_from_root(consequence,nx_grapht,node)<2:
    #                 if node==2753:
    #                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
    #                     print(list(nx_grapht.neighbors(node)))
                    anastomosis.append(node)
                    origins.append(tip)
                    number_anastomosis+=1
            if tip not in nx_grapht.nodes and dist_branch(tip,nx_grapht,pos_tm1)<=30 and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['weight']>=20:
                origins.append(tip)
                number_anastomosis+=1/2
        return(anastomosis,origins,number_anastomosis)
    def find_image_pos(self,ys,xs,t,local=False):
        params=read_mat(self.path_param)
        x1=params['x1']
        x2=params['x2']
        y0=params['y0']
        Sf=params['Sf']
        L0center=params['L0center']
        date = self.dates[t]
        date_plate = f'/2020{date}'
        plate_str = f'_Plate{self.plate}'
        findingbaits = read_mat('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Analysis/FindingBaits.mat')
        Rcenter=findingbaits['Rcenter']
        Rradii = findingbaits['Rradii']
        Lcenter = findingbaits['Lcenter']
        t = findingbaits['t']
        Lradii = findingbaits['Lradii']
        path_tile='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
        xs_yss=list(tileconfig[2])
        xes= [xs_ys[0] for xs_ys in xs_yss]
        yes= [xs_ys[1] for xs_ys in xs_yss]
        cmin=np.min(xes)
        cmax=np.max(xes)
        rmin = np.min(yes)
        rmax = np.max(yes)
        S0     = [rmax+3000+abs(rmin),cmax+4096+abs(cmin)]
        theta  = np.arctan((Rcenter[1]-Lcenter[1])/(Rcenter[0]-Lcenter[0]))
        Sry    =  S0[0]*np.cos(abs(theta))+S0[1]*np.sin(abs(theta))
        C = np.tan(-np.arctan((ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/
                              (xs + (x2+x1)/2 - (Sf[0]-y0) - 1 - L0center[0] + np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                               *np.cos(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))) - theta)
        ximg = - (ys + y0 - 1 - L0center[1] - np.sqrt(Lcenter[0]**2+(S0[0]-Lcenter[1])**2)
                  *np.sin(np.arctan((S0[0]-Lcenter[1])/Lcenter[0])+theta))/(np.sqrt(1+C**2)*np.sin(np.arctan(C)+theta)) 
        yimg = S0[0] - C*ximg
        def find(xsub,ysub,x,y):
            indexes=[]
            for i in range(len(xsub)):
                if x>=xsub[i]-cmin and x<xsub[i]-cmin+4096 and y>=ysub[i]-rmin and y < ysub[i]-rmin+3000:
                    indexes.append(i)
            return(indexes)
        indsImg = find(xes,yes,ximg,yimg)
        possImg  = [ximg-np.array(xes)[indsImg]+cmin+1, yimg-np.array(yes)[indsImg]+rmin+1]
        paths=[]
        for index in indsImg:
            if local:
                paths.append('Temp/'+tileconfig[0][index]+'.npy')
            else:
                paths.append('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+tileconfig[0][index])
        if local:
             ims = [np.load(path) for path in paths]
        else:
            ims = [imageio.imread(path) for path in paths]
        return(ims,possImg)
    def plot(self,ts,node_lists=[], shift=(0,0),compress=5,save='',time=None):
        global check
        right = 0.90
        top = 0.90
        if len(save)>=1:
            fig=plt.figure(figsize=(14,12))
            size = 10
        else:
            fig = plt.figure()
            size = 5
        ax = fig.add_subplot(111)
        scale=1/len(ts)
        power=len(ts)
        compressed_images=[]
        for i,t in enumerate(ts):
            compressed_images.append(self.compress_skeleton(t,compress))
        visibility = [True for t in ts]
        final_image=scale*compressed_images[0]*visibility[0]
        for i,compressed_image in enumerate(compressed_images[1:]):
            final_image=np.power(final_image**power+((i+2)*scale*compressed_image*visibility[i+1])**power,1/power)
        l1=ax.imshow(final_image, cmap='plasma',interpolation='none',vmin=0,vmax=1)
        rax = plt.axes([0.05, 0.4, 0.1, 0.15])
        labels=[f'{4*t}h' for t in ts]
        check = CheckButtons(rax, labels, visibility)
        def func(label):
            index = labels.index(label)
            visibility[index]= not visibility[index]
            final_image=visibility[0]*scale*compressed_images[0]
            for i,compressed_image in enumerate(compressed_images[1:]):
                final_image=np.power(final_image**power+visibility[i+1]*((i+2)*scale*compressed_image)**power,1/power)
            l1.set_data(final_image)
            plt.draw()

        check.on_clicked(func)
        if len(node_lists)>0:
            for i,node_list in enumerate(node_lists):
                grey=(i+1)/len(labels)
                bbox = dict(boxstyle="circle", fc=colors.rgb2hex((grey,grey,grey)))
#             ax.text(right, top, time,
#                 horizontalalignment='right',
#                 verticalalignment='bottom',
#                 transform=ax.transAxes,color='white')
                for node in node_list:
#                     print(self.positions[ts[i]])
                    if node in self.positions[ts[i]].keys():
                        t = ax.text((self.positions[ts[i]][node][1]-shift[1])//compress, (self.positions[ts[i]][node][0]-shift[0])//compress, str(node), ha="center", va="center",
                                    size=size,
                                    bbox=bbox)
        if len(save)>=1:
            plt.savefig(save)
            plt.close(fig)
        else:
            plt.show()
        
        

class Node():
    def __init__(self,label,experiment):
        self.experiment = experiment
        self.label = label
    def __eq__(self, other): 
        return(self.label==other.label)
    def __repr__(self):
        return (f'Node({self.label})')
    def __str__(self):
        return str(self.label)
    def __hash__(self):
        return self.label
    def neighbours(self,t):
        return([self.experiment.get_node(node) for node in self.experiment.nx_graph[t].neighbors(self.label)])
    def is_in(self,t):
        return(self.label in self.experiment.nx_graph[t].nodes)
    def degree(self,t):
        return(self.experiment.nx_graph[t].degree(self.label))
    def edges(self,t):
        return([self.experiment.get_edge(self,neighbour) for neighbour in self.neighbours(t)])
    def pos(self,t):
        return(self.experiment.positions[t][self.label])
    def ts(self):
        return([t for t in range(len(self.experiment.nx_graph)) if self.is_in(t)])
    def show_source_image(self,t):
        pos = self.pos(t)
        x,y=pos[0],pos[1]
        ims,posimg=self.experiment.find_image_pos(x,y,t)
        i=1
        plot_t_tp1([0],[],{0 : (posimg[1][i],posimg[0][i])},None,ims[i],ims[i])

class Edge():
    def __init__(self,begin,end,experiment):
        self.begin=begin
        self.end = end
        self.experiment = experiment
    def __repr__(self):
        return (f'Edge({self.begin},{self.end})')
    def __str__(self):
        return str((self.begin,self.end))
    def is_in(self,t):
        return((self.begin.label,self.end.label) in self.experiment.nx_graph[t].edges)
    def pixel_list(self,t):
        return(orient(self.experiment.nx_graph[t].get_edge_data(self.begin.label,self.end.label)['pixel_list'],self.begin.pos(t)))
    def orientation_whole(self,t):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
    def orientation_begin(self,t,length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[min(length,len(pixel_list)-1)]-pixel_list[0]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
    def orientation_end(self,t,length=20):
        pixel_list = np.array(self.pixel_list(t))
        vector = pixel_list[-1]-pixel_list[max(0,len(pixel_list)-1-length)]
        unit_vector = vector/np.linalg.norm(vector)
        vertical_vector=np.array([-1,0])
        dot_product = np.dot(vertical_vector,unit_vector)
        if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
            angle = np.arccos(dot_product)/(2*np.pi)*360
        else:
            angle = -np.arccos(dot_product)/(2*np.pi)*360
        return(angle)
        

class Hyphae():
    def __init__(self,tip):
        self.experiment = tip.experiment
        self.ts = tip.ts()
        self.end = tip
        self.root = None
        self.mother = None
    def __repr__(self):
        return (f'Hyphae({self.end},{self.root})')
    def __str__(self):
        return str((self.end,self.root))
    def get_edges(self,t,length=20):
        first_neighbour = self.end.neighbours(t)[0]
        llast_node = self.end
        ccurrent_node = first_neighbour
        def move_hyphae(last_node,current_node):
            current_edge = Edge(last_node,current_node,current_node.experiment)
            moving_on_hyphae = True
            edges = [current_edge]
            nodes = [last_node,current_node]
            while moving_on_hyphae:
#                 print ('moving',current_node)
                if current_node.degree(t)<2:
#                     print(current_node.degree(t),current_node)
                    return(edges,nodes,current_node)
                else:
                    maxi=-np.inf
                    orientation = current_edge.orientation_end(t,length)
                    for neighbours_t in current_node.neighbours(t):
    #                     print (neighbours_t)
                        candidate_edge = Edge(current_node,neighbours_t,current_node.experiment)
                        orientation_candidate = candidate_edge.orientation_begin(t,length)
                        angle = np.cos((orientation-orientation_candidate)/360*2*np.pi)
                        if angle>maxi:
                            maxi=angle
                            next_node_candidate=neighbours_t
#                     print(maxi,next_node_candidate)
                    candidate_edge = Edge(current_node,next_node_candidate,current_node.experiment)
                    orientation_candidate = candidate_edge.orientation_begin(t,length)
                    maxi_compet=-np.inf
#                     print('compet')
                    for neighbours_t in current_node.neighbours(t):
                        if neighbours_t!=last_node:
                            competitor_edge = Edge(neighbours_t,current_node,current_node.experiment)
                            orientation_competitor = competitor_edge.orientation_end(t,length)
                            angle = np.cos((orientation_competitor-orientation_candidate)/360*2*np.pi)
                            if angle>maxi_compet:
                                maxi_compet=angle
                                competitor = neighbours_t
#                             print(neighbours_t,angle)
#                     print(maxi_compet,competitor)
                    if maxi_compet>maxi:
                        moving_on_hyphae =False
                    else:
                        last_node,current_node = current_node,next_node_candidate
                        current_edge = Edge(last_node,current_node,current_node.experiment)
                        edges.append(current_edge)
                        nodes.append(current_node)
            return(edges,nodes,competitor)
        moving = True
        edges = []
        nodes = []
        c= move_hyphae(llast_node,ccurrent_node)
        edges += c[0]
        nodes += c[1]
        competitor = c[2]
#         while moving:
#             c= move_hyphae(llast_node,ccurrent_node)
#             edges += c[0]
#             nodes += c[1]
#             competitor = c[2]
# #             print('moving back', nodes[-1],competitor)
#             move_backward = move_hyphae(nodes[-1],competitor)
#             end_node_move_backward = move_backward[1][-1]
#             if end_node_move_backward in nodes:
# #                 print('restarting',competitor,nodes[-1])
#                 llast_node,ccurrent_node = c[2],nodes[-1]
#             else:
#                 moving=False
        root = nodes[-1]
        edges = edges
        nodes = nodes
        return(root,edges,nodes)
    def get_nodes_within(self,t):
        nodes = nx.shortest_path(self.experiment.nx_graph[t],source=self.root.label,target=self.end.label,weight="weight")
        edges = [Edge(self.experiment.get_node(nodes[i]),self.experiment.get_node(nodes[i+1]),self.experiment) for i in range(len(nodes)-1)]
        return(nodes,edges)
    def get_mother(self):
        candidate_mother=[]
        for hyphae in self.experiment.hyphaes:
            for t in hyphae.ts:
                if self.root in hyphae.get_nodes_within(t):
                    candidate_mother.append(hyphae)
        self.mother = candidate_mother

def get_hyphae(experiment):
    tips = [node for node in experiment.nodes if node.degree(node.ts()[0])==1]
    problems=[]
    hyphaes=[]
#     for i in range(20):
    for i,tip in enumerate(tips):
        if i%200==0:
            print(i/len(tips))
#         tip = choice(tips)
        hyphae = Hyphae(tip)
        roots = []
        for t in tip.ts():
#             print(t,tip)
            if tip.degree(t)==1:
                root,edges,nodes = hyphae.get_edges(t,50)
                roots.append(root)
        occurence_count = Counter(roots)
        if len(occurence_count.values())>=2 and occurence_count.most_common(2)[0][0]!=roots[0] and occurence_count.most_common(2)[1][1]/occurence_count.most_common(2)[0][1]>=0.75:
#             print(roots[0])
#             print(tip,occurence_count)
            problems.append(tip)
        else:
            hyphae.root = occurence_count.most_common(2)[0][0]
            hyphaes.append(hyphae)
    return(hyphaes,problems)

def reconnect_degree_2(nx_graph,pos):
    degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]
    while len(degree_2_nodes)>0:
        node = degree_2_nodes.pop()
        neighbours = list(nx_graph.neighbors(node))
        right_n = neighbours[0]
        left_n = neighbours[1]
        right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
        left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
        if np.any(right_edge[0]!=pos[node]):
            right_edge = list(reversed(right_edge))
        if np.any(left_edge[-1]!=pos[node]):
            left_edge = list(reversed(left_edge))
        pixel_list = left_edge+right_edge[1:]
        info={'weight':len(pixel_list),'pixel_list':pixel_list}
        if right_n!=left_n:
            connection_data=nx_graph.get_edge_data(right_n,left_n)
            if connection_data is None or connection_data['weight']>=info['weight']:
                if not connection_data is None:
                    nx_graph.remove_edge(right_n,left_n)
                nx_graph.add_edges_from([(right_n,left_n,info)])
        nx_graph.remove_node(node)
        degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]

def clean_exp_with_hyphaes(experiment):
    ts={}
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
    exp_clean = Experiment(experiment.plate)
    exp_clean.copy(experiment)
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes=[]
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    for node in experiment.nodes:
        ts[node.label]=[]
        for t in range(len(nx_graph_cleans)):
            if node.is_in(t):
                ts[node.label].append(t)
    hyphaes,problems = get_hyphae(exp_clean)
    to_remove=[]
    safe_nodes = set()
    for hyphae in hyphaes:
        for t in hyphae.end.ts():
            if t in hyphae.root.ts():
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if len(ts[node.label])==1 and ts[node.label][0]!=len(experiment.nx_graph)-1 and node.label not in safe_nodes:
            to_remove.append(node.label)
#     return(to_remove)
    print('removing ',len(to_remove),' nodes')
    for node in to_remove:
        t= ts[node][0]
        pos = experiment.positions[t]
        nx_graph_clean = nx_graph_cleans[t]
        if nx_graph_clean.degree(node)<=2:
            continue
#         print('cleaning',node)
        neighbours = list(nx_graph_clean.neighbors(node))
        candidate_to_fuse=[]
        weight_candidate=[]
        for neighbour in neighbours:
            if nx_graph_clean.degree(neighbour)>=3:
                candidate_to_fuse.append(neighbour)
                weight_candidate.append(len(nx_graph_clean.get_edge_data(node,neighbour)['pixel_list']))
        if len(weight_candidate)==0:
            continue
        node_to_fuse=candidate_to_fuse[np.argmin(weight_candidate)]
        if nx_graph_clean.degree(node)==1 and node_to_fuse not in to_remove:
            print(pos[node])
            continue
        for neighbour in neighbours:
            right_n = node_to_fuse
            left_n = neighbour
            right_edge = nx_graph_clean.get_edge_data(node,right_n)['pixel_list']
            left_edge = nx_graph_clean.get_edge_data(node,left_n)['pixel_list']
            if np.any(right_edge[0]!=pos[node]):
                right_edge = list(reversed(right_edge))
            if np.any(left_edge[-1]!=pos[node]):
                left_edge = list(reversed(left_edge))
            pixel_list = left_edge+right_edge[1:]
            info={'weight':len(pixel_list),'pixel_list':pixel_list}
            if right_n!=left_n:
                connection_data=nx_graph_clean.get_edge_data(right_n,left_n)
                if connection_data is None or connection_data['weight']>=info['weight']:
                    if not connection_data is None:
                        nx_graph_clean.remove_edge(right_n,left_n)
                    nx_graph_clean.add_edges_from([(right_n,left_n,info)])
        nx_graph_clean.remove_node(node)
        nx_graph_cleans[t]=nx_graph_clean
    for t,nx_graph in enumerate(nx_graph_cleans):
        pos = experiment.positions[t]
        reconnect_degree_2(nx_graph,pos)
    nx_graph_clean=[]
    for graph in nx_graph_cleans:
        S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
        len_connected=[len(nx_graph.nodes) for nx_graph in S]
        nx_graph_clean.append(S[np.argmax(len_connected)])
    skeletons=[]
    for nx_graph in nx_graph_clean:
        skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))
    exp_clean.nx_graph=nx_graph_clean
    exp_clean.skeletons=skeletons
    labels = {node for g in exp_clean.nx_graph for node in g}
    exp_clean.nodes=[]
    exp_clean.hyphaes = hyphaes
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    return(exp_clean)